# Laboratoire 3
# SVM, ensembles et apprentissage non-supervisé
## Classification, régression et régroupement (FER et FG-NET)

### GTI771 - Apprentissage machine avancé
#### Département du génie logiciel et des technologies de l’information

#### Version 1.0 mars 2020
#### <font color=blue> Version 2.0 - 22 mars 2020 </font>
#### <font color=magenta> Version 3.0 - 3 avril 2020 </font>

##### Prof. Alessandro L. Koerich

| Étudiants             | Branavan Inthirnathan - INTB25099502                    |
|-----------------------|---------------------------------------------------------|
| Session               | HIV 2020                                            |
| Équipe                | 07                                                       |
| Numéro du laboratoire | 3                                                       |
| Professeur            | Prof. Alessandro L. Koerich                                               |
| Chargé de laboratoire |  Alessandro L. Koerich                                                     |
| Date                  | 04/19/20                                                    |

# Introduction

Ce troisième laboratoire porte sur l’utilisation des machines à vecteurs de support (SVM) et régresseurs à vecteurs de support (SVR), la combinaison de modèles d’apprentissage et le regroupement.

Dans ce laboratoire, vous êtes amenés à utiliser de nouvelles approches à l’aide de ces algorithmes/approches afin de résoudre le problème de classification des émotions (FER dataset) et le problème de régression des âges (FG-NET dataset). De plus, dans ce laboratoire, vous êtes amené aussi à considérer les algorithmes d’apprentissage non supervisé.

Vous devrez utiliser les primitives développées aux premiers laboratoires lorsque celles-ci ne sont pas redondantes. Vous devrez également étudier les hyperparamètres du modèle. Tout comme les deux premiers laboratoires de ce cours, vous réaliserez ce troisième travail avec la technologie Python3 conjointement avec la librairie d’apprentissage machine scikit-learn.

Votre Jupyter notebook devra contenir les réponses aux questions. Il devra notamment avoir une analyse détaillée des résultats de classification obtenus par les différents modèles et leurs variations d’hyperparamètres.

L’évaluation de ce laboratoire sera basée sur la qualité des modèles entraînés, la comparaison des performances réalisées par les différents modèles, les réponses aux questions dans cette notebook ainsi que l’organisation de votre code source (SVP, n’oubliez pas des commentaires dans le code!).

# Objectifs

Les principaux objectifs de ce laboratoire sont :
1. Produire un code source utilisant les technologies Python et des techniques d’apprentissage machine permettant de classifier des échantillons de données automatiquement;
2. Se familiariser avec les algorithmes SVM et SVR;
3. Analyser l’impact des hyperparamètres et des différents noyaux utilisés;
4. De construire un véritable système intelligent avec une combinaison de modèles (et/ou primitives);
5. Se familiariser avec les algorithmes d’apprentissage non supervisé;
6. Analyser et interpréter les sorties des algorithmes non supervisés;
7. Se familiariser avec le langage de programmation Python et les outils et librairies scientifiques d’apprentissage machine;

* #### Partie 1: SVM - Classification et régression
* SVM lineaire
* SVM noyau polynomial
* SVM noyau RBF
<font color=blue>
* SVR lineaire ou SVR noyau polynomial ou SVR noyau RBF
</font>    
<br>
* #### Partie 2: Ensembles
<font color=blue>
* Bagging, Boosting / Adaboost
* Votation, Min, Max, Average, Product, Weighted Rules
</font>        
<br>
* #### Partie 3: Regroupement
<font color=magenta>
* k-Means
* Autre méthode de regroupement
</font>    

## Partie 0: Imports

### (0a) Import de libraries

##### À faire:
1. Ajouter toutes les bibliothèques que vous avez utilisées pour compléter ce notebook dans une cellule avec une petite description.

In [ ]:
import numpy as np  # package for scientific computing with Python.
import matplotlib.pyplot as plt # 2D plotting library
from sklearn.preprocessing import MinMaxScaler #Normalizer
import os 
import math
import csv
from sklearn import svm
from hypopt import GridSearch #hyperparametre optimization
from sklearn.tree import DecisionTreeClassifier #classifier 2b
from sklearn.ensemble import RandomForestClassifier #classifier 2b
from sklearn.neighbors import KNeighborsClassifier#classifier 2b
from sklearn.naive_bayes import GaussianNB #classifier 2b
from sklearn.ensemble import VotingClassifier #classifier 2b
from sklearn.model_selection import GridSearchCV #classifier 2b for grid search
from sklearn.metrics import accuracy_score
from joblib import dump, load #export les modèles
from hypopt import GridSearch #hyperparametre optimization
#import thundersvm
from sklearn import tree
###### Clusters ##########
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering 
from sklearn.cluster import Birch

from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import adjusted_rand_score

# Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


### (0b) Charger le fichier de données

##### À faire:
1. Ajouter le code pour lire les fichiers de données.

In [ ]:
pip install thundersvm

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load data
ferData = np.loadtxt('fer2013.csv', delimiter=',', dtype=np.str)
Xtrain = np.ones((28709,2304),np.uint8)
for i in range(1, 28710):
	Xtrain[i-1]=ferData[i,1].split(" ")

ytrain=ferData[1:28710,0].astype(np.int)
Xval = np.ones((3589,2304),float)
for i in range(28710, 32299):
	Xval[i-28710]=ferData[i,1].split(" ")

yval=ferData[28710:32299,0].astype(np.int)
Xtest = np.ones((3589,2304),float)
for i in range(32299, 35888):
	Xtest[i-32299]=ferData[i,1].split(" ")

ytest=ferData[32299:,0].astype(np.int)

# normalize
scaler = MinMaxScaler(feature_range=(0, 1))

Xtrain = scaler.fit_transform(Xtrain)
Xval   = scaler.fit_transform(Xval)
Xtest  = scaler.fit_transform(Xtest)

# Partie 1: Machines à vecteurs de support (SVM)
## (FER et FG-NET dataset)

Dans cette partie vous devez explorer les <b> différentes noyaux du SVM </b> disponibles dans LIBSVM, comme linéaire, polynomial et RBF.

Vous devez comparer la performance de ces algorithmes pour les ensemble FER (classification) et FG-NET (régression)

## (1a) Créer et évaluer des modèles SVM de classification
### (FER dataset)

[Scikit-learn LibSVM](https://scikit-learn.org/stable/modules/svm.html)

##### À faire:
1. Vecteurs de primitives << artisanales >> (handcrafted sans/avec réduction) du laboratoire 1. Choisir une seule représentation (p. ex. LBP après PCA). N’oubliez pas de normaliser les vecteurs (entrées) entre 0 et 1. 
2. Vecteurs de primitives << deep >> (CNN features sans/avec réduction) du laboratoire 2. Choisir une seule représentation (p. ex. 1re couche entièrement connectée du CNN simple). Vous pouvez également appliquer un algorithme de réduction de la dimensionnalité pour réduire la dimension de ce vecteur. N’oubliez pas de normaliser les vecteurs (entrées) entre 0 et 1.
3. Pour chaque représentation (artisanale et deep):<br> 
3a. Entraîner et optimiser (C) un SVM linéaire <br>
3b. Entraîner et optimiser (C et ordre) un SVM polynomial <br>
3c. Entraîner et optimiser (C et gamma) un SVM RBF (Gaussian) <br>
4. Analyser les résultats et présenter vos conclusions sur les modèles SVM.

|  Primitive   | Noyau       |   Paramètres    |  % App | % Val  | % Tst  | 
|--------------|-------------|-----------------|--------|--------|--------|
| LBP avec PCA | lineaire    | C=256            | 19.123 | 18.501 | 19.170       |
| LBP avec PCA | polynomial  | C=10,d=6         |  17.311 |16.968        | 17.414       |
| LBP avec PCA | RBF         | C=256, g=0.01   | 18.865 | 18.807 | 17.414 |
| CNN 5L 2FC     | lineaire    | C=10            | 25.37 | 16.912 |  17.024      |
| CNN 5L 2FC     | polynomial  | C=10,d=3         |  76.683 | 14.712       |  16.299      |
| CNN 5L 2FC     | RBF         | C=10,d=3,g=0.05   | 91.946 |16.188 | 18.278     |

###Analyse des résultats###

Avec l'utilisation de primitives des laboratoires précédant, soit le vecteur artisanal de LBP avec l'application de l'algorithme de transformation et le vecteur deep avec le CNN de 5 couches convolutifs et 2 couches entièrement connectées, dans les modèles SVM de classification. On peut observer selon les résultats des différents vecteurs dans les modèles SVM que la précision de ces primitives est sensiblement la même, en effet avec la primitive LBP/PCA, elle donne entre 17% et 19% pour  l'ensemble d'entrainement, de validation et test. Par la suite, avec la primitive CNN, elle donne entre 15% et 18% pour l'ensemble de validation et de test avec une précision sur l'ensemble d'apprentissages assez élevé 25%, 76 % et 92%. Donc, les modèles avec la primitive LBP/PCA obtiennent du résultat plus élevé avec 18.5% en validation et 19.2% en test pour un noyau SVM linéaire, 16.97% en validation et 17.4% avec un noyau SVM polynomial et 18.8% en validation et 17.4% avec un noyau SVM rbf. Pour ce qui est des modèles avec la primitive CNN, la meilleure précision est 16.2% en validation et 18.3% en test pour un noyau SVM rbf, mais quand même un surapprentissage à 91.9% de précision sur l'ensemble d'apprentissages, sinon il est suivi par le modèle SVM avec un noyau linière dont la précision est de 16.9% sur l'ensemble de validation et 17% sur l'ensemble de tests, puis ça termine avec le modèle avec un noyau polynomial pour une précision de 14.7% sur validation et 16.3% sur test.

### (1a): Code:

#### HandCrafted features

In [ ]:
# Votre code ici
# FER primitives LBP
"""PCA"""
#LBP
PCA_lbp_vector_app = np.genfromtxt('PCA_lbp_vector_app.csv', delimiter=';')
PCA_lbp_vector_val = np.genfromtxt('PCA_lbp_vector_val.csv', delimiter=';')
PCA_lbp_vector_test = np.genfromtxt('PCA_lbp_vector_test.csv', delimiter=';')

#Normalizer vector
scaler = MinMaxScaler(feature_range=(0, 1))
"""PCA"""
PCA_lbp_vector_app = scaler.fit_transform(PCA_lbp_vector_app)
PCA_lbp_vector_val = scaler.fit_transform(PCA_lbp_vector_val)
PCA_lbp_vector_test = scaler.fit_transform(PCA_lbp_vector_test)

#### CNN features with normalization and reduced Dimension on CNN 5L 2FC

In [ ]:
PCA_Xflat_features_app = np.load("pca_deep_vector_app.npy")
PCA_Xflat_features_val   = np.load("pca_deep_vector_val.npy")
PCA_Xflat_features_test  = np.load("pca_deep_vector_test.npy")

FileNotFoundError: ignored

#### Reduce Dimension -- Don't Run 

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# Xflat_features_app = np.load("FER_Xflat_features_train.npy")
# Xflat_features_val   = np.load("FER_Xflat_features_val.npy")
# Xflat_features_test  = np.load("FER_Xflat_features_test.npy")

# def pcaTrans(vector_train):
#     pca = PCA(n_components=100)
#     principalComponents = pca.fit_transform(vector_train)
#     pca_vector = pca.transform( vector_train )
#     return principalComponents

# PCA_Xflat_features_app = pcaTrans(Xflat_features_app)
# PCA_Xflat_features_test = pcaTrans(Xflat_features_test)
# PCA_Xflat_features_val = pcaTrans(Xflat_features_val)

In [ ]:
# np.save( "pca_deep_vector_app.npy", PCA_Xflat_features_app)
# np.save( "pca_deep_vector_val.npy", PCA_Xflat_features_test)
# np.save( "pca_deep_vector_test.npy", PCA_Xflat_features_val)

In [ ]:
#Methode pour faire les différentes SVM necessaires

def linearSVM_model():
    print("SVM with Linear Kernel (one-versus-one)\n")
    model = svm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
                    max_iter=4000, probability=True, random_state=None, shrinking=False,
                    tol=0.001, verbose=True)
    return model

def polySVM_model():
    print("SVM with Polynomial Kernel (one-versus-one)\n")
    model = svm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='poly',
                    max_iter=4000, probability=True, random_state=None, shrinking=False,
                    tol=0.001, verbose=True)
    return model

def rbfSVM_model():
    print("SVM with Gaussian Kernel(one-versus-one)\n")
    model = svm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
                    max_iter=4000, probability=True, random_state=None, shrinking=False,
                    tol=0.001, verbose=True)
    return model

#### GPU version

In [ ]:
def linearSVM_model():
    print("SVM with Linear Kernel (one-versus-one)\n")
    model = thundersvm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
                    max_iter=4000, probability=True, random_state=None, shrinking=False,
                    tol=0.001, verbose=True)
    return model

def polySVM_model():
    print("SVM with Polynomial Kernel (one-versus-one)\n")
    model = thundersvm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='poly',
                    max_iter=4000, probability=True, random_state=None, shrinking=False, 
                    tol=0.001, verbose=True)
    return model

def rbfSVM_model():
    print("SVM with Gaussian Kernel(one-versus-one)\n")
    model = thundersvm.SVC(C=10.0, cache_size=2000, class_weight=None, coef0=0.0,
                    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
                    max_iter=4000, probability=True, random_state=None, shrinking=False,
                    tol=0.001, verbose=True)
    return model

In [ ]:
# Vos résultats ici:
#Permet de faire plusieurs tests de hyperparametres
param_grid_lin = [{'kernel': ['linear'], 'C': [10,50,100 256], 'max_iter': [4000]}]
param_grid_poly = [{'kernel': ['poly'], 'C': [10, 50,100,256], 'max_iter': [4000], 'degree':[3,6, 10]}]
param_grid_rbf = [{'kernel': ['rbf'], 'C': [10, 256], 'max_iter': [4000], 'gamma':[0.01, 0.05, 0.1]}]

SyntaxError: ignored

### (1a): Résultats et résponses:

In [ ]:
from sklearn.metrics import accuracy_score
def showResultat(tuned_model, vector) :
    print('The best performing parameters and their scores.')
    for z in tuned_model.get_param_scores()[:2]:
        p, s = z
        print(p)
        print('Score:', s)
    print('Verify that the lowest scoring parameters make sense.')
    for z in tuned_model.get_param_scores()[-2:]:
        p, s = z
        print(p)
        print('Score:', s)
    if (vector == "handCrafted"):
        # Use the tuned model to predict the class labels
        Ytrain_pred_tun = tuned_model.predict(PCA_lbp_vector_app)
        Yval_pred_tun   = tuned_model.predict(PCA_lbp_vector_val)
        Ytest_pred_tun  = tuned_model.predict(PCA_lbp_vector_test)

    else:
        Ytrain_pred_tun = tuned_model.predict(Xflat_features_app)
        Yval_pred_tun   = tuned_model.predict(Xflat_features_val)
        Ytest_pred_tun  = tuned_model.predict(Xflat_features_test)
        
    # Final evaluation of the model (On the Training, Validation or Test dataset)
    scores_tun = accuracy_score(ytrain, Ytrain_pred_tun )
    print("Correct classification rate for the training dataset (tuned_model)   = "+str(scores_tun*100)+"%")

    scores2_tun = accuracy_score(yval, Yval_pred_tun )
    print("Correct classification rate for the validation dataset (tuned_model) = "+str(scores2_tun*100)+"%")

    scores3_tun = accuracy_score(ytest, Ytest_pred_tun )
    print("Correct classification rate for the test dataset (tuned_model) = "+str(scores3_tun*100)+"%")

In [ ]:
 # Final evaluation of the model (On the Training, Validation or Test dataset)
    scores_tun = accuracy_score(ytrain, Ytrain_pred_tun )
    print("Correct classification rate for the training dataset (tuned_model)   = "+str(scores_tun*100)+"%")

    scores2_tun = accuracy_score(yval, Yval_pred_tun )
    print("Correct classification rate for the validation dataset (tuned_model) = "+str(scores2_tun*100)+"%")

    scores3_tun = accuracy_score(ytest, Ytest_pred_tun )
    print("Correct classification rate for the test dataset (tuned_model) = "+str(scores3_tun*100)+"%")

NameError: ignored

In [ ]:
tuned_model_lin = GridSearch(linearSVM_model(), param_grid=param_grid_lin)
model_lin_deep = tuned_model_lin

%time tuned_model_lin.fit(PCA_lbp_vector_app,ytrain)
%time model_lin_deep.fit(Xflat_features_app, ytrain)

NameError: ignored

In [ ]:
tuned_model_poly = GridSearch(polySVM_model(), param_grid=param_grid_poly)
model_poly_deep = tuned_model_poly

%time tuned_model_poly.fit(PCA_lbp_vector_app,ytrain)
%time model_poly_deep.fit(Xflat_features_app,ytrain)

In [ ]:
tuned_model_rbf = GridSearch(rbfSVM_model(), param_grid=param_grid_rbf)
model_rbf_deep = rbfSVM_model

%time tuned_model_rbf.fit(PCA_lbp_vector_app,ytrain)
%time model_rbf_deep.fit(Xflat_features_train, ytrain)

#### Results for handcrafted vectors

In [ ]:
showResultat(tuned_model_lin, "handCrafted")

In [ ]:
showResultat(tuned_model_poly, "handCrafted")

In [ ]:
showResultat(tuned_model_rbf, "handCrafted")

#### Results for Deep vectors

In [ ]:
showResultat(model_lin_deep, "deep")

In [ ]:
showResultat(model_poly_deep, "deep")

In [ ]:
showResultat(model_rbf_deep, "deep")

## (1b) Créer et évaluer des modèles de régression
### (FG-NET dataset)

Idéalement, vous devez tout refaire pour ce nouvel ensemble de données / tâche. Alors, pour simplifier ce TP, vous pouvez créer, optimiser et évaluer un seul modèle de régression. Vous pouvez choisir la meilleure combinaison représentation / SVM de la partie 1a. 
<font color=blue>
##### À faire:
1. Choisir une combinaison primitive/kernel (Cellui qu'a meilleur performé dans la partie 1). 
2. Entraîner et optimiser (C, epsilon, ???) un SRV (linéaire ou polynomial ou Gaussian)
3. Analyser les résultats et présenter vos conclusions sur le modèle SVR.

|  Primitive   | Noyau    |   Paramètres       |  MAE   |  MSE   | 
|--------------|----------|--------------------|--------|--------|
| FG_NET |  Lin     | C=100,ep=2               |8.286   |  144.864      |
</font>    

###Analyse des résultats###

Dans cette partie, la combinaison choisie pour effectuer le modèle SVR est en utilisant comme jeu de primitive le FG_NET 256 par 256 et comme kernel le linéaire. Le modèle SVR avec Kernel linéaire ajuste les coefficients de poids pour minimiser le résidu de somme des carrés entre cibles observées et prédites, avec comme paramètres un C=100 et epsilon de 2, le meansquarederror est de 144.864et la meanabsoluteerror est de 8.286 pour faire les prédictions. Les résultats pour ce modèle SVR montrent des erreurs moins élevées qu'au précédent avec l'algorithme de régression linéaire ordinaire.

### (1b): Code:

#### Preprocess data -- Taken from exemple 

In [ ]:
X_data = np.loadtxt('fgnet_256x256.csv', delimiter=',', dtype=int)
Y_data = np.loadtxt('fgnet_labels.csv', delimiter=',', dtype=int)
Y_subj = np.loadtxt('fgnet_subjects.csv', delimiter=',', dtype=int)

X_data = X_data.reshape(X_data.shape[0], 1, 256, 256).astype('uint8')

# Adapted from Equipe 1 - Julien Robitaille and Bengoufa Mohamed Abdeldjalil
from skimage.feature import local_binary_pattern
import pathlib

data_to_LBP = [
    (X_data, Y_data)
]
def format_nb(x: int):
    return str(int(x))

X_data_lbp = np.zeros((1002 ,59))

# Adapted from Equipe 1 - Julien Robitaille and Bengoufa Mohamed Abdeldjalil
#Local binary pattern feature to csv
#local_binary_pattern(image, P, R, method='default')
P= 8 
R= 2

lbp_dim = local_binary_pattern(np.squeeze(X_data[0], axis=0), P, R, 'nri_uniform')
nb_bins = int(lbp_dim.max() + 1)

number_of_points = 59

X_data_lbp = np.zeros((1002 ,number_of_points))

for data in data_to_LBP:
    x, y = data
    for i, image in enumerate(x):
        lbp = local_binary_pattern(image[0], P, R, 'nri_uniform') #Local binary pattern extraction
        hist, _ = np.histogram(lbp.ravel(), density=True, bins=nb_bins ,range=(0, nb_bins))
        hist = hist.astype(np.float)
        hist /= (hist.sum() + 1e-7)
        X_data_lbp[i,:] =  hist
        
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
X_data_lbp = scaler.fit_transform(X_data_lbp)

X_data_lbp.shape, Y_data.shape, Y_subj.shape

In [ ]:
def linearSVR_model():
    print("SVR with Linear Kernel\n")
    model = thundersvm.SVR(epsilon=0.1, C=10.0, cache_size=2000,
                    degree=2, kernel='linear',
                    max_iter=-1, shrinking=False,
                    tol=0.0001, verbose=True)
    return model

In [ ]:
# Assuming you already have training, validation and test sets, as well as a "model".
from hypopt import GridSearch
param_grid = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000], 'epsilon': [0.01, 0.1, 0.2, 0.25, 0.5, 1, 2, 5]}]


In [ ]:
model       = linearSVR_model()
tuned_model = GridSearch(model = model, param_grid = param_grid)

%time tuned_model.fit(X_data_lbp, Y_data, scoring="neg_mean_absolute_error", scoring_params = None, verbose=True )

In [ ]:
# Use the model to predict just the class labels
Y_data_pred = tuned_model.predict(X_data_lbp)

# Compute the MAE and MSE metrics for Regression
from sklearn.metrics import mean_squared_error, mean_absolute_error

def MSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    # print('MSE: %2.5f' % mse)
    return mse

def MAE(y_true,y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    # print('MAE: %2.5f' % mse)
    return mae

# Final evaluation of the model (On the Training, Validation or Test dataset)
scores = MAE( Y_data, Y_data_pred )
print("MAE for the training dataset   = "+str(scores)+" years")

In [ ]:
print('The best performing parameters and their scores.')
for z in tuned_model.get_param_scores()[:1]:
    p, s = z
    print(p)
    print('Score:', s)
print()

### (1b): Résultats et résponses:

In [ ]:
# Use the tuned model to predict the class labels
Y_data_pred_tun = tuned_model.predict(X_data_lbp)
# Final evaluation of the model (On the Training, Validation or Test dataset)
scores_tun = MAE(Y_data, Y_data_pred_tun  )
mse =  MSE(Y_data, Y_data_pred_tun)

print("MAE for the dataset (model)       = "+str(scores)+" years")
print("MAE for the dataset (tuned_model) = "+str(scores_tun)+" years")
print("MSE for the dataset (tuned_model) = "+str(mse)+" years")

### (1)Analyse

Voici les résultats obtenus pour l'algorithme de régression avec (FG-NET dataset) ainsi que pour les algorithmes de classification (FER dataset) dans la partie 1 du laboratoire.

|  Primitive   | Noyau    |   Paramètres       |  MAE   |  MSE   | Comparaison | Primitive   | Noyau       |   Paramètres    |  % App | % Val  | % Tst  | 
|--------------|----------|--------------------||--------|--------|--------------|-------------|-----------------|--------|--------|--------|
| FG_NET |  Lin     | C=100,ep=2               |8.286   |  144.864       |----| LBP avec PCA | lineaire    | C=256            | 19.123 | 18.501 | 19.170       |
||||||----| LBP avec PCA | polynomial  | C=10,d=6         |  17.311 |16.968        | 17.414       |
||||||----| LBP avec PCA | RBF         | C=256, g=0.01   | 18.865 | 18.807 | 17.414 |
||||||----| CNN 5L 2FC     | lineaire    | C=10            | 25.37 | 16.912 |  17.024      |
||||||----| CNN 5L 2FC     | polynomial  | C=10,d=3         |  76.683 | 14.712       |  16.299      |
||||||----| CNN 5L 2FC     | RBF         | C=10,d=3,g=0.05   | 91.946 |16.188 | 18.278     |

Pour ce qui est de la partie 1 avec la régression sur les données FG-NET, les principaux résultats qu'on peut observer sont avec la régression SVR avec un noyau linéaire dont la MSE obtenue est de 144.864 et MAE obtenue est de 8.286. Puis, pour la partie avec la classification sur les données FER, avec les primitives LBP avec transformation PCA, les principaux résultats avec le modèle SVM sont avec une précision entre 17% et 19% avec les noyaux linéaire, polynomial et rbf et ceux avec les vecteurs de primitives CNN sont proche avec une précision entre 15% et 18%, toutefois l'erreur reste assez élevée pour les modèles. 

# Partie 2: Combinaison des algorithmes d'apprentissage
## (FER dataset)

Vous devrez intégrer une notion supplémentaire acquise récemment dans le cours théorique, à savoir la combinaison de modèles d’apprentissage. Contrairement aux laboratoires précédents dans lesquelles vous travailliez avec des modèles indépendants sur un seul ensemble de données, vous devez dans ce présent laboratoire combiner des algorithmes d’apprentissage afin de classifier les échantillons d’un ensemble de données regroupant plusieurs sortes de primitives (feature sets). Vous implémenterez ce système avec les librairies TensorFlow ou scikit-learn. Finalement, vous aurez à proposer une stratégie combinatoire de classificateurs tel que vote de majorité, moyenne, produit, médiane, etc.

## (2a) Créer et évaluer des ensembles Bagging et Boosting

<font color=blue>
    
##### À faire:
1. Choisir un algorithme de base (AD, NB, KNN) pour construire des ensembles avec "bagging" et "boosting"
2. Entraîner et optimiser les ensembles bagging et Adaboost
3. Comparer la performance des ensembles avec des classificateurs individuels.     
4. Analyser les résultats et présenter vos conclusions sur les ensembles bagging et boosting.

|  Primitive   | Classificateur | Ensemble |  paramètres |  % App | % Val  | % Tst  | 
|--------------|----------------|----------|-------------|--------|--------|--------|
| LBP avec PCA |       AD       | bagging  |ms=0.4,ne=50 | XXX.XX | XXX.XX | XXX.XX |
| LBP avec PCA |       AD       | adaboost |lr=1.0,ne=200| XXX.XX | XXX.XX | XXX.XX |    
| ...          |                |                        | XXX.XX | XXX.XX | XXX.XX |    
</font>    








## **Analyse des résultats**


Le Bagging (Bootstrap Aggregation) et l'Adaboost sont des techniques d'optimisation utilisées pour améliorer la classification notamment celle avec des arbres de décision, qui representaient des classificateurs très faibles. 
Le** Bagging** a pour but de ** réduire la variance** de l’estimateur alors que **l'Adaboost** c'est pour **minimiser le biais** de l'estimateur.
Neanmoins, dans notre cas la performance de ces deux techniques étaient à peu près la meme chose vu la faible dimension de nos vecteurs primitives(LBP avec PCA) ; apres avoir optimiser les hyperparametres sur l'ensemble de validatiion,on obtient pour un classificateur AD avec un bagging  25.55% pour les données d'apprentissage contre 25.59% pour AD avec un Adaboost 
et 24,85% pour les données test pour le bagging contre 23.26% pour l'Adaboost.



L'utilisation de ces deux ensembles dans notre cas n'a pas trop augmenter la performance de notre AD individuel(25.61% pour les données d'apprentissage  et 24.77% pour les données test) et cela est expliqué par **la faible dimension de nos vecteurs primitives ce qui n'assure pas suffisament de diversité.**




<font color=black>

|  Primitive   | Classificateur | Ensemble |  paramètres |  % App | % Val  | % Tst  | 
|--------------|----------------|----------|-------------|--------|--------|--------|
| LBP avec PCA |       AD       | bagging  |ms=0.4,mf=0.5,ne=50,nj=8 |25.55%|25.35%|24.85%|
| LBP avec PCA |       AD       | adaboost |lr=1.0,ne=10|25.59%  | 22.73% | 23.26%|    
|   LBP avec PCA       |     AD individuel           |                        | |25.61%|25.10% | 24.77% |
| LBP avec PCA |       NB individuel    |  || 25.27% | 22.09% | 22.48% | 
| LBP avec PCA |       KNN individuel      |  || 44,99% | 18.97% | 20.78% |
| LBP avec PCA |       Random Forest    |  |  |32.06% | 18.61% | 19.0025%


<front>

Généralement les ensembles (Bagging et Adaboost) avec un AD et plus performant sur les données test qu'aux autres classificateurs individuels tel est le cas de KNN(20.78%), NB(22.48%),RF(19.0025%)

### <font color=blue> (2a) Code: </font> 

In [ ]:
pip install hypopt

In [ ]:
"On choisit les hyperparametres par une 'cross-validation' pour optimiser notre algorithme d'Arbre de décision"
# Set the parameters by cross-validation
# Assuming you already have train, test, val sets and a model.
from hypopt import GridSearch

tuned_parameters = [{'max_depth': [2, 4, 6], 
                     'min_samples_leaf': [10,20 ], 
                     'min_samples_split': [5, 10, 20]}]

#DecisionTreeClassifier(criterion='gini', max_depth=4 ,min_samples_leaf=10, random_state=1)                     

In [ ]:
"Un Grid search pour trouver les parametres qui optimise notre modele "
# Grid-search all parameter combinations using a validation set.

tuned_model = GridSearch(model = tree.DecisionTreeClassifier(criterion='entropy'),param_grid = tuned_parameters)
tuned_model.fit(PCA_lbp_vector_app, ytrain, PCA_lbp_vector_val, yval)

print("Best parameters set found on validation set:")
print()
print('Test Score for Optimized Parameters:', tuned_model.score(PCA_lbp_vector_val, yval)) #

100%|██████████| 18/18 [00:01<00:00, 12.17it/s]

Best parameters set found on validation set:

Test Score for Optimized Parameters: 0.251044859292282


In [ ]:
print('We can view the best performing parameters and their scores.')
for z in tuned_model.get_param_scores()[:2]:
    p, s = z
    print(p)
    print('Score:', s)
print()
print('Verify that the lowest scoring parameters make sense.')
for z in tuned_model.get_param_scores()[-2:]:
    p, s = z
    print(p)
    print('Score:', s)

We can view the best performing parameters and their scores.
{'max_depth': 4, 'min_samples_leaf': 10, 'min_samples_split': 20}
Score: 0.251044859292282
{'max_depth': 4, 'min_samples_leaf': 10, 'min_samples_split': 5}
Score: 0.251044859292282

Verify that the lowest scoring parameters make sense.
{'max_depth': 6, 'min_samples_leaf': 20, 'min_samples_split': 5}
Score: 0.20897185845639454
{'max_depth': 6, 'min_samples_leaf': 20, 'min_samples_split': 20}
Score: 0.20897185845639454


In [ ]:
scores_tuned  = accuracy_score(ytrain, Ytrain_pred )
scores2_tuned = accuracy_score(yval, Yvalid_pred)
scores3_tuned = accuracy_score(ytest, Ytest_pred )
print(scores_tuned ,scores2_tuned ,scores3_tuned )

0.2561566059423874 0.251044859292282 0.24770130955697967


In [ ]:
# Use the tunes model to predict the class of samples

Ytrain_pred = tuned_model.predict(PCA_lbp_vector_app)
Yvalid_pred = tuned_model.predict(PCA_lbp_vector_val)
Ytest_pred  = tuned_model.predict(PCA_lbp_vector_test)

In [ ]:
model = DT_model() # Build the model
model.fit(PCA_lbp_vector_app,ytrain) # Fit the model (TRAIN)
Ytrain_pred = model.predict(PCA_lbp_vector_app)#predict the class of samples
Yvalid_pred = model.predict(PCA_lbp_vector_val)
Ytest_pred  = model.predict(PCA_lbp_vector_test)                    

**BAGGING**

In [ ]:
"On utilise un AD pour construire un ensemble BAGGING"
#BAGGING
from sklearn.ensemble import BaggingClassifier
# Using a bagging of Decision Trees

#bagging = BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth=15, min_samples_leaf=5, min_samples_split=10),
bagging = BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=10, random_state=1),
                            max_samples  = 0.4,
                             max_features = 0.5,
                            n_estimators = 50,
                            n_jobs = 8,
                            bootstrap = False)
#model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=10, random_state=1)

In [ ]:
# Fit the model (TRAIN)
bagging.fit(PCA_lbp_vector_app,ytrain)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=1,
  

In [ ]:

# Use the model to predict the class of samples

Ytrain_pred_bagging = bagging.predict(PCA_lbp_vector_app)
Yvalid_pred_bagging = bagging.predict(PCA_lbp_vector_val)
Ytest_pred_bagging  = bagging.predict(PCA_lbp_vector_test)

**AdaBoost**

In [ ]:
"On construit avec le classificateur AD un ensemble AdaBoost"
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Using a AdaBoosting of Decision Trees

#AdaB= AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth=15, min_samples_leaf=5, min_samples_split=10),
AdaB= AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth=3
                                                     , min_samples_leaf=10, random_state=1), 
                         n_estimators = 10, 
                         learning_rate = 1)

In [ ]:
# Fit the model to the AdaB (TRAIN)

AdaB.fit(PCA_lbp_vector_app,ytrain)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                      

In [ ]:
# Use the model to predict the class of samples
# Notice that we are testing the train dataset

Ytrain_pred_adaboost = AdaB.predict(PCA_lbp_vector_app)

Yvalid_pred_adaboost = AdaB.predict(PCA_lbp_vector_val)
Ytest_pred_adaboost  = AdaB.predict(PCA_lbp_vector_test)

**`Autres classificateurs individuels`**

In [ ]:
# Training classifiers
clf2 = RandomForestClassifier(n_estimators=50, max_depth=12, min_samples_split=2, random_state=1)
clf3 = GaussianNB()
clf4 = KNeighborsClassifier(n_neighbors=20,weights = 'uniform',metric = 'euclidean', algorithm='kd_tree')

### <font color=blue> (2a): Résultats et résponses: </font> 


In [ ]:
#On compare la performance des classificateurs AD , AD+Bagging et AD+AdaBoost

'''classificateur AD individuel '''
scores = accuracy_score(ytrain, Ytrain_pred )
scores2 = accuracy_score(yval, Yvalid_pred )
scores3 = accuracy_score(ytest, Ytest_pred )


'''classificateur AD + BAGGING '''
scores_bagging  = accuracy_score(ytrain, Ytrain_pred_bagging)
scores2_bagging = accuracy_score(yval, Yvalid_pred_bagging)
scores3_bagging = accuracy_score(ytest, Ytest_pred_bagging)


'''classificateur AD+AdaBoost'''
scores_adaboost  = accuracy_score(ytrain, Ytrain_pred_adaboost )
scores2_adaboost = accuracy_score(yval, Yvalid_pred_adaboost )
scores3_adaboost = accuracy_score(ytest, Ytest_pred_adaboost )


print("Correct classification rate for the training dataset (AD individuel)      = "+str(scores*100)+"%")
print("Correct classification rate for the training dataset (AD+bagging)     = "+str(scores_bagging*100)+"%")
print("Correct classification rate for the training dataset (AD+adaboost)    = "+str(scores_adaboost*100)+"%")
print()
print()
print("Correct classification rate for the validation dataset (AD individuel)      = "+str(scores2*100)+"%")
print("Correct classification rate for the validation dataset (AD+bagging)     = "+str(scores2_bagging*100)+"%")
print("Correct classification rate for the validation dataset (AD+adaboost)    = "+str(scores2_adaboost*100)+"%")
print()
print()
print("Correct classification rate for the test dataset (AD individuel)      = "+str(scores3*100)+"%")
print("Correct classification rate for the test dataset (AD +bagging)     = "+str(scores3_bagging*100)+"%")
print("Correct classification rate for the test dataset (AD +adaboost)    = "+str(scores3_adaboost*100)+"%")

Correct classification rate for the training dataset (AD individuel)      = 25.615660594238744%
Correct classification rate for the training dataset (AD+bagging)     = 25.552962485631685%
Correct classification rate for the training dataset (AD+adaboost)    = 25.598244452959%


Correct classification rate for the validation dataset (AD individuel)      = 25.104485929228197%
Correct classification rate for the validation dataset (AD+bagging)     = 24.993034271384786%
Correct classification rate for the validation dataset (AD+adaboost)    = 22.736138200055724%


Correct classification rate for the test dataset (AD individuel)      = 24.770130955697965%
Correct classification rate for the test dataset (AD +bagging)     = 25.43884090275843%
Correct classification rate for the test dataset (AD +adaboost)    = 23.265533574811926%


In [ ]:
# Fit the Ensemble (TRAIN)
# Sert seulement à l'évaluation les performances d classificateurs simples
clf2.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf2.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf2.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf2.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (RT)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (RT)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (RT)      = "+str(scores_tst_ENSEMBLE*100)+"%")
clf3.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf3.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf3.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf3.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (NB)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (NB)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (NB)      = "+str(scores_tst_ENSEMBLE*100)+"%")
clf4.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf4.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf4.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf4.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (KNN)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (KNN)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (KNN)      = "+str(scores_tst_ENSEMBLE*100)+"%")


Classification rate for the training dataset (RT)      = 44.99285938207531%
Classification rate for the validation dataset  (RT)     = 18.974644747840625%
Classification rate for the test dataset (RT)      = 20.78573418779604%

Classification rate for the training dataset (NB)      = 25.277787453411822%
Classification rate for the validation dataset  (NB)     = 22.095291167456114%
Classification rate for the test dataset (NB)      = 22.48537196990805%

Classification rate for the training dataset (KNN)      = 32.06659932425372%
Classification rate for the validation dataset  (KNN)     = 18.612426859849542%
Classification rate for the test dataset (KNN)      = 19.002507662301475%


## <font color=blue> (2b) Créer et évaluer des combinaison de modèles </font>

<font color=blue>
    
##### À faire:

1. Réalisez une combinaison de au moins 3 modèles d’apprentissage (p.ex. AD, NB et kNN). Pour ce faire, vous pouvez
combiner des modèles précédemment élaborés ou en générer d’autres. Au moins 3
modèles doivent faire partie de votre combinaison.
2. Choisir au moins deux règles de combinaison. Votre règle de décision peut être inspirée des notes de cours ou de la librairie scikit-learn et de sa documentation, pourvu que vous justifiiez le choix de celle-ci.
3. Produisez un diagramme afin d’illustrer votre stratégie d’ensemble qui permet clairement de voir quels ensembles de primitives vous avez choisis d’inclurent, quels classificateurs font partie de votre solution et quelle stratégie de combinaison vous avez choisie. Joignez-le à ce notebook.

<img src="picture.png" style="height:600px">
    
4. Exportez vos modèles. Cet exemple vous aidera à exporter vos modèles à l’aide de la librairie scikit-learn, alors que ce lien vous aidera avec TensorFlow.
5. Comparer la performance des ensembles avec des classificateurs individuels et des ensembles (partie 1).     
6. Analyser les résultats et présenter vos conclusions sur les combinaisons de modèles.
</font>


Utilisation du VotingClassifier de la librairie scikit-learn pour faire la combinaison de modèles selon deux différentes règles soit __le vote par majorité__ et __la votation pondérée sur les classes prédites__, le jeu de primitives PCA_lbp est le vecteur de données pour faire l'évaluation de ce classificateur par votation avec quatre différents modèles de classification, soit le DecisionTreeClassifier, le randomForestClassifier, le gaussian naives bayes et KNeighborsClassifier.

|JEU DE PRIMITIVES| CLASSIFICATEUR |COMBINAISON |RÉSULTATS|
|-|-|-|-|                   
||                   ---->DT------>
||                   ---->RT------>
|PCA_lbp -------->|================|--->VotingClassifier--->|y
||                   ---->NB------>
||                   ---->nKK----->

|JEU DE PRIMITIVES| CLASSIFICATEUR |COMBINAISON |RÉSULTATS|
|-|-|-|-|                   
||                   ---->DT------>(1.8)
||                   ---->RT------>(0.8)
|PCA_lbp -------->|================|--->VotingClassifier--->|y
||                   ---->NB------>(1.2)
||                   ---->nKK----->(0.8)

La performance par les six classificateurs individuels de la partie 1 sont moins élevé avec des précisions obtenues entre 16% et 18% pour l'ensemble validation et l'ensemble test, tandis que les ensembles obtiennent au moins une précision de 22% et 24% pour les ensembles de validation et de test, ce qui représente une différence d'au moins 4% entre les classificateurs simples et des ensembles.

|  Primitive   | Noyau       |   Paramètres    |  % App  |  % Val | % Tst | _VS_ |  Primitive   | Classificateurs |   Règle  |  paramètres |  % App | % Val  | % Tst  |
|--------------|-------------|-----------------|---------|--------|--------|---|--------------|------------------|---------|-------------|--------|---------|--------|
| LBP avec PCA | lineaire    | C=256            | 19.123 | 18.501 | 19.170 |----| LBP avec PCA |  DT, RT, NB, kNN | votation majorité |'hard' | 31.282 | 22.597 | 22.820 |
| LBP avec PCA | polynomial  | C=10,d=6         |  17.311 |16.968 | 17.414 |----| LBP avec PCA |  DT, RT, NB, kNN | votation pondérée |'hard''1.8,0.8,1.2,0.8'| 27.214 | 24.463 | 24.408 |
| LBP avec PCA | RBF         | C=256, g=0.01    | 18.865 | 18.807 | 17.414 |----|--------------|------------------|--------------------| |--------|---------|--------|
| CNN 5L 2FC   | lineaire    | C=10             | 25.37 | 16.912 | 17.024 |----|--------------|------------------|--------------------| |--------|---------|--------|
| CNN 5L 2FC   | polynomial  | C=10,d=3         |  76.683 | 14.712 | 16.299|----|--------------|------------------|--------------------| |--------|---------|--------|
| CNN 5L 2FC   | RBF         | C=10,d=3,g=0.05  | 91.946_|16.188_| 18.278_|----|--------------|------------------|--------------------||--------|---------|--------|

Avec le premier modèle de combinaisons fait avec comme paramètres la votation par majorité(soit voting='hard') et le jeu de primitives LBP avec PCA, les résultats que nous pouvons avoir sont un pourcentage d'efficacité de 31.28% pour l'ensemble d'entrainement, un pourcentage d'efficacité de 22.60% pour l'ensemble de validation et un pourcentage d'efficacité de 22.82% pour l'ensemble de test. Ce qui est pour le deuxième modèle avec comme paramètres la votation par majorité avec des poids pondérés(soit voting='hard',weights=[1.8,0.8,1.2,0.8]), nous avons obtenu un pourcentage de 27.21% pour l'ensemble d'entrainement, un pourcentage de 24.46% pour l'ensemble de validation et un pourcentage de 24.41% pour l'ensemble de test. Donc, avec les résultats de deux classificateurs d'ensembles, on remarque que la précision obtenue pour les ensembles de validation et de test est 2% plus élevé pour celui dont la règle de votation pondérée est appliquée.

###  (2b): Code:

In [ ]:
# Training classifiers
clf1 = DecisionTreeClassifier(criterion='gini', max_depth=4 ,min_samples_leaf=10, random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, max_depth=12, min_samples_split=2, random_state=1)
clf3 = GaussianNB()
clf4 = KNeighborsClassifier(n_neighbors=20,weights = 'uniform',metric = 'euclidean', algorithm='kd_tree')
# Training ensemble
model_votation_majority = VotingClassifier(estimators=[('dt', clf1), ('rf',clf2),('nb', clf3), ('knn', clf4)], voting='hard', n_jobs=-1)

model_predict_proba_votation = VotingClassifier(estimators=[('dt', clf1), ('rf',clf2),('nb', clf3), ('knn', clf4)], voting='hard',weights=[1.8,0.8,1.2,0.8], n_jobs=-1)

#params = {'dt__max_depth': [10,20,40,50], 'dt__min_samples_leaf': [2,10,20], 'dt__min_samples_split': [2,10,20], 'rf__n_estimators': [10, 200], 'rf__max_depth': [10,20,40,50] ,'rf__min_samples_split': [2, 10, 20]}
#grid = GridSearchCV(estimator=model_votation_majority, param_grid=params, cv=5)
#grid.best_params_
model_votation_majority = model_votation_majority.fit(PCA_lbp_vector_app, ytrain)
# Use the Combinaison model to predict the class of samples
Ytrain_pred_Votation = model_votation_majority.predict(PCA_lbp_vector_app)
Yvalid_pred_Votation = model_votation_majority.predict(PCA_lbp_vector_val)
Ytest_pred_Votation  = model_votation_majority.predict(PCA_lbp_vector_test)

model_predict_ponderation_votation = model_predict_proba_votation.fit(PCA_lbp_vector_app, ytrain)
# Use the Combinaison model to predict the class of samples
Ytrain_pred_predict_proba = model_predict_ponderation_votation.predict(PCA_lbp_vector_app)
Yvalid_pred_predict_proba = model_predict_ponderation_votation.predict(PCA_lbp_vector_val)
Ytest_pred_predict_proba  = model_predict_ponderation_votation.predict(PCA_lbp_vector_test)

dump(model_votation_majority, 'model_votation_majority_fit.joblib')
dump(model_predict_ponderation_votation, 'model_ponderation_votation.joblib')

['model_ponderation_votation.joblib']

###  (2b): Résultats et résponses:

In [ ]:
# Vos résultats ici:
scores_tr_ENSEMBLE  = accuracy_score(ytrain, Ytrain_pred_Votation )
scores_val_ENSEMBLE = accuracy_score(yval, Yvalid_pred_Votation )
scores_tst_ENSEMBLE = accuracy_score(ytest,  Ytest_pred_Votation )
print("Classification rate for the training dataset (Ensemble_MAJORITY)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (Ensemble_MAJORITY)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (Ensemble_MAJORITY)      = "+str(scores_tst_ENSEMBLE*100)+"%")
# Vos résultats ici:
# ENSEMBLE
scores_tr_ENSEMBLE  = accuracy_score(ytrain, Ytrain_pred_predict_proba )
scores_val_ENSEMBLE = accuracy_score(yval, Yvalid_pred_predict_proba )
scores_tst_ENSEMBLE = accuracy_score(ytest,  Ytest_pred_predict_proba )
print()
print("Classification rate for the training dataset (Ensemble_PREDICT_PROBA)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (Ensemble_PREDICT_PROBA)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (Ensemble_PREDICT_PROBA)      = "+str(scores_tst_ENSEMBLE*100)+"%")

Classification rate for the training dataset (Ensemble_MAJORITY)      = 31.282872966665504%
Classification rate for the validation dataset  (Ensemble_MAJORITY)     = 22.596823627751466%
Classification rate for the test dataset (Ensemble_MAJORITY)      = 22.819726943438283%

Classification rate for the training dataset (Ensemble_PREDICT_PROBA)      = 27.214462363718695%
Classification rate for the validation dataset  (Ensemble_PREDICT_PROBA)     = 24.463638896628588%
Classification rate for the test dataset (Ensemble_PREDICT_PROBA)      = 24.407913067706883%


In [ ]:
# Fit the Ensemble (TRAIN)
# Sert seulement à l'évaluation les performances des classificateurs simples mis dans le VotingClassifier
clf1.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf1.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf1.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf1.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (DT)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (DT)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (DT)      = "+str(scores_tst_ENSEMBLE*100)+"%")
clf2.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf2.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf2.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf2.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (RT)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (RT)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (RT)      = "+str(scores_tst_ENSEMBLE*100)+"%")
clf3.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf3.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf3.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf3.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (NB)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (NB)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (NB)      = "+str(scores_tst_ENSEMBLE*100)+"%")
clf4.fit(PCA_lbp_vector_app, ytrain)
scores_tr_ENSEMBLE  = accuracy_score(ytrain, clf4.predict(PCA_lbp_vector_app) )
scores_val_ENSEMBLE = accuracy_score(yval, clf4.predict(PCA_lbp_vector_val) )
scores_tst_ENSEMBLE = accuracy_score(ytest,  clf4.predict(PCA_lbp_vector_test) )
print()
print("Classification rate for the training dataset (KNN)      = "+str(scores_tr_ENSEMBLE*100)+"%")
print("Classification rate for the validation dataset  (KNN)     = "+str(scores_val_ENSEMBLE*100)+"%")
print("Classification rate for the test dataset (KNN)      = "+str(scores_tst_ENSEMBLE*100)+"%")


Classification rate for the training dataset (DT)      = 25.845553659131284%
Classification rate for the validation dataset  (DT)     = 25.299526330454164%
Classification rate for the test dataset (DT)      = 25.13234884368905%

Classification rate for the training dataset (RT)      = 44.99285938207531%
Classification rate for the validation dataset  (RT)     = 18.974644747840625%
Classification rate for the test dataset (RT)      = 20.78573418779604%

Classification rate for the training dataset (NB)      = 25.277787453411822%
Classification rate for the validation dataset  (NB)     = 22.095291167456114%
Classification rate for the test dataset (NB)      = 22.48537196990805%

Classification rate for the training dataset (KNN)      = 32.06659932425372%
Classification rate for the validation dataset  (KNN)     = 18.612426859849542%
Classification rate for the test dataset (KNN)      = 19.002507662301475%


# Partie 3: Régroupement
## (FER dataset)


Dans cette partie, nous allons ignorer les tâches <font color=magenta>de classification ou régression</font> les et les étiquettes. Nous allons supposer que les données sont non supervisées (non étiquetées).

Le but est de découvrir l’organisation des données, c.-à-d. découvrir les similitudes et les différences entre les données et d’en tirer des conclusions utiles (trouver des groupes cohérents).

Les tâches de regroupement
<font color=magenta>
- Identifier des groupes « naturels » dans les données;
- Discretiser un espace $\mathcal{R}^D$, en le séparant en $K$ régions. C'est equivalent à partitionner l’espace d’entrée en un certain nombre de « catégories » en se basant sur un ensemble d’apprentissage fini.
</font>


<font color=black>

### Partie (3a): Code régroupement (FER):
    
##### À faire :
1. Utiliser les algorithmes de regroupement de [scikit-learn clustering](https://scikit-learn.org/stable/modules/clustering.html#clustering)
2. Choisir deux algorithmes de regroupement <br>
    <font color=magenta>Vous devez également regarder le tableau [Overview of clustering methods](https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods) pour voir quels sont les algorithmes de regroupement les plus appropriés en fonction des caractéristiques de l'ensemble de données FER (surtout « scalability » et « usecase »). (P.ex. $K$-means et hierarchical agglomerative. Voir le notebook exemple.)<br>
3. Entraîner et optimiser les paramètres des algorithmes de regroupement choisis. Utiliser l'ensemble de validation pour cette optimisation.<br>
4. Utiliser également l'ensemble de validation pour interpréter les résultats du regroupement. En regardant quelques exemples de chaque groupe, pouvons-nous assigner une étiquette fiable pour chaque groupe? P.ex. « Le groupe 0 représente l'émotion contente. Le groupe 1 représente l'émotion triste, etc. ».<br>
5. Faire une analyse des résultats et présenter vos conclusions sur l'interprétation des groupes trouvées. <br>
 - Analyser les groupes en supposant que les vraies étiquettes sont inconnues. <br>
 - Analyser les groupes en supposant que les vraies étiquettes sont connues.
     - Attention avec la consistance des étiquettes, c.-à-d., vous devez vous assurer d'avoir la bonne codification, chiffre = émotion.<br>
     - Dans le ground-truth nous avons 0 = Angry, 1 = Disgust, 2 = Fear, 3 = Happy, 4 = Sad, 5 = Surprise, 6 = Neutral, mais probablement vous n'allez pas avoir nécessairement cette même codification comme sortie d'algorithme de regroupement.
        
</font>    
</font>

In [ ]:
image_shape = [48, 48]

X_people = Xtrain
y_people = ytrain
X_people = X_people / 255

### PCA

In [ ]:
def plot_pca(labels):
    X_pca_df = pd.DataFrame(data = X_pca)
    ## 2 first PCA components
    fig = plt.figure()
    fig.set_size_inches(20,10)
    plt.scatter(X_pca_df.iloc[:, 0], X_pca_df.iloc[:, 1],
                c = labels, edgecolor='none', alpha=0.5,
                cmap = plt.cm.get_cmap('Spectral', 7))
    plt.xlabel('component 1')
    plt.ylabel('component 2')
    plt.colorbar();
    
    from mpl_toolkits import mplot3d

    ## 3 first PCA components

    fig = plt.figure()
    fig.set_size_inches(20,10)
    ax = plt.axes(projection="3d")

    p = ax.scatter3D(X_pca_df.iloc[:, 0], X_pca_df.iloc[:, 1], X_pca_df.iloc[:, 2],
                     c = labels, cmap = 'Spectral')

    ax.set_xlabel('component 1')
    ax.set_ylabel('component 2')
    ax.set_zlabel('component 3')
    fig.colorbar(p)
    plt.show()

In [ ]:
pca = PCA( n_components = 100 )
pca.fit_transform( X_people )
X_pca = pca.transform( X_people )

#### Plot PCA

In [ ]:
plot_pca(y_people)

#### Performance Evaluation

In [ ]:
def grouth_truth_eval(model_labels):
    # Perfect labeling is scored 1.0
    print("Adjusted Rand index: ", metrics.adjusted_rand_score(y_people, model_labels) )

    # Perfect labeling is scored 1.0
    print("Mutual Information based scores: ", metrics.adjusted_mutual_info_score(y_people, model_labels) )

    # Both are bounded below by 0.0 and above by 1.0 (higher is better)
    print("Homogeneity: ", metrics.homogeneity_score(y_people, model_labels) )
    print("Completeness: ", metrics.completeness_score(y_people, model_labels) )
    print("V-Measure: ", metrics.v_measure_score(y_people, model_labels) )
    print("Fowlkes-Mallows scores: ", metrics.fowlkes_mallows_score(y_people, model_labels) )

def non_grouth_eval(model_labels):
    # The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
    # The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
    print("Silhouette coefficient: ", metrics.silhouette_score(X_pca, model_labels , metric='euclidean') )

    # The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
    print("Calinski-Harabasz index: ", metrics.calinski_harabasz_score(X_pca, model_labels) )

    # Zero is the lowest possible score. Values closer to zero indicate a better partition.
    print("Davies-Bouldin Index: ", metrics.davies_bouldin_score(X_pca, model_labels) )

### BIRCH


In [ ]:
birch = Birch(threshold=0.005, n_clusters=7, compute_labels=True, copy=True)

clusters = birch.fit_predict(X_pca)
            
print("cluster sizes Birch: %s" % np.bincount(clusters))

core_samples_mask = np.zeros_like(birch.labels_)
labels = birch.labels_

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
n_clusters_

In [ ]:
# Utilise pour comparer avec les vrais etiquettes
plot_pca(labels)

In [ ]:
clusters = birch.fit_predict(X_pca)
grouth_truth_eval(clusters, y_people)
non_grouth_eval(clusters, X_pca)

In [ ]:
clusters = birch.predict(X_val_pca)
grouth_truth_eval(clusters, yval)
non_grouth_eval(clusters, X_val_pca)

In [ ]:
clusters = birch.fit_predict(X_test_pca)
grouth_truth_eval(clusters, ytest)
non_grouth_eval(clusters, X_test_pca)


#### Optimisation


In [ ]:
for n_cluster in range(2, 11):
    spec = Birch(n_clusters=n_cluster, threshold=0.005).fit(X_val_pca)
    label = spec.labels_
    sil_coeff = silhouette_score(X_val_pca, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

In [ ]:
for n_cluster in range(2, 6):
    spec = Birch(n_clusters=n_cluster, threshold=0.005, branching_factor=10*n_cluster).fit(X_val_pca)
    label = spec.labels_
    sil_coeff = silhouette_score(X_val_pca, label, metric='euclidean')
    print("For branching factor={}, The Silhouette Coefficient is {}".format(10*n_cluster, sil_coeff))

### DBSCAN


In [ ]:
# covariance = np.cov(X_pca.astype("float32"), rowvar=False)
# db = DBSCAN(eps=0.1, min_samples=15, algorithm='ball_tree')
# db = DBSCAN(eps=0.5, min_samples=5, algorithm='ball_tree', metric='minkowski', leaf_size=90, p=2)
# db = DBSCAN(min_samples=6, eps=3, metric="mahalanobis", metric_params={"V": covariance})

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=0.03, min_samples=10, algorithm='ball_tree', metric='minkowski', leaf_size=90, p=2)
clusters = db.fit_predict(X_pca)

core_samples_mask = np.zeros_like(db.labels_)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
n_clusters_

In [ ]:
grouth_truth_eval(clusters, y_people)
non_grouth_eval(clusters, X_pca)

In [ ]:
clusters = db.fit_predict(X_val_pca)
grouth_truth_eval(clusters, ytest)
non_grouth_eval(clusters, X_val_pca)

In [ ]:
clusters = db.fit_predict(X_test_pca)
grouth_truth_eval(clusters, ytest)
non_grouth_eval(clusters, X_test_pca)

In [ ]:
plot_pca(labels)

#### Optimisation


In [ ]:
# for eps_op in [0.01, 0.02, 0.03, 0.05]:
#     spec = DBSCAN(eps=eps_op, min_samples=10).fit(X_val_pca)
#     label = spec.labels_
#     sil_coeff = silhouette_score(X_val_pca, label, metric='euclidean')
#     print("For eps={}, Epsilan is {}".format(eps_op, sil_coeff))

In [ ]:
# for min_op in [5, 10, 15,  20, 30, 50]:
#     spec = DBSCAN(eps=0.03, min_samples=min_op).fit(X_val_pca)
#     label = spec.labels_
#     sil_coeff = silhouette_score(X_val_pca, label, metric='euclidean')
#     print("For min_sample={}, min sample is {}".format(min_op, sil_coeff))

Ces deux cellules ne fonctionnent pas, mais ils sont présent à titre indicative pour montrer quels valeurs à optimiser. 

### Analyse 


### Partie (3a): Résultats et résponses:

#### Votre analyse et résultats ici:

<font color=black>
    
##### Birch (évaluation sans étiquettes):
| Ensemble | AR index | MI score | Homo | Completeness | V-measure | F-M score |
|----------|----------|----------|------|--------------|-----------|-----------|
| App      | 0.008     | 0.012     |0.014 | 0.014   | 0.014    | 0.184 |
| Val      | 0.013     | 0.012    | 0.014 | 0.0178  |  0.016    | 0.217 |
| Test     | 0.012    | 0.014     | 0.014 |  0.017     |   0.0178    | 0.0159 |


##### Birch (évaluation avec étiquettes):
| Ensemble | Sillouette | C-H score | D-B index |
|----------|------------|-----------|-----------|
| App      |  0.378  | 2639.364     | 2.758 |
| Val      |  0.053  | 320.110     | 2.95 |
| Test     | 0.0625   | 358.59     | 2.5974 |


</font>    

 Pour que les algorithmes de regroupement fonctionnent correctement, il nous faut 
avant tout des groupes de données discernables. En entrée, nous avons les données FER où nous avons implémenté PCA à 100 composants pour diminuer la taille de données des vecteurs. Lorsqu'on trace en x,y et z, les trois premiers composants PCAs avec les 
étiquettes réelles. Il est possible de voir que toutes les émotions sont confondues. Il est impossible de distinguer des zones de concentrations d'émotions distinctes.Si nous regardons  les clusters créés par notre algorithme, il est possible de voir que la zone
-0.1 à -0.025 de l'axe du composant 1, il y a de l'émotion 0, vers la zone 0.02 à 0.06, il a plus de l'émotion 2 et 3, etc. Cette division claire des émotions est issue de l'algorithme et ne  reflète pas la réalité comme il est possible de le voir par notre évaluation avec étiquettes.En effet, par exemple, notre ajusted rand index de 0.012, alors qu'un résultat parfait serait 1.  Pour notre évaluation sans étiquette, nous remarquons que nos clusters ne sont bien définis(silhouette coefficient). Les clusters sont très denses. Cela est normal, car notre threshold est très petit. Et notre D-B index n'est pas à zéro, mais elle est proche. Cela indique une bonne partition des clusters. 

Pour optimiser notre algorthme Birch, nous avons fait varier le nombre de clusters et notre branching factor. Nous avons gardé un threshold bas de 0.005, car nos points sont très proches et un threshold élevé cause des erreurs potentielles avec créant pas assez de clusters. 
Dans chacun des cas, nous voulons trouver un coefficient de silhouette élevé. Selon la documentation de sklearn, un coefficient élevé nous dit que notre modèle possède des clusters mieux définis. Pour le nombre de clusters, 2 clusters nous donnent le mieux coefficient. Or étant donné que nous avons sept classes, deux clusters ne sont simplement pas suffisants. Le nombre de clusters qui est mieux est 5, avec un coefficient de 0.083. Ces résultats ne sont certainement pas élevés, mais c'est le mieux que nous avions pû trouvé. Pour le branching factor, nous avons trouvé que 20 est le meilleur choix. Un branching factor plus faible (le nombre par défaut est de 50)nous permet d'avoir une meilleure séparation des donnés.


##### DBSCAN (évaluation sans étiquettes):
| Ensemble | AR index | MI score | Homo | Completeness | V-measure | F-M score |
|----------|----------|----------|------|--------------|-----------|-----------|
| App      | 0.006     | 0.006     |0.005 | 0.014   | 0.007    | 0.32 |
| Val      | X.XXX     | X.XXX    | X.XXX | X.XXX           |  X.XXX    | X.XXX |
| Test     | X.XXX    | X.XXX     | X.XXX |  X.XXX     |   X.XXX    | X.XXX |


##### DBSCAN (évaluation avec étiquettes):
| Ensemble | Sillouette | C-H score | D-B index |
|----------|------------|-----------|-----------|
| App      |  -0.12  | 22.312     | 3.748 |
| Val      |  X.XXX  | X.XXX     | X.XXX |
| Test     | X.XXX   | X.XXX     | X.XXX |

Pour DBSCAN, il y a deux paramètres à faire varier pour changer le nombre et dimension des clusters produites: epsilon et min_samples. Après de nombreux essais, nous n'avons pas trouvés de modèle adéquat pour regrouper nos dataset val et test pour FER. En effet, en diminuant notre epsilon et notre min_sample,nous voulons créer plusieurs clusters. Or en ce faisant, nous avions des dizaines voire des centaines de clusters, mais seulement un cluster pour nos données val et test. Lorsqu'on trace les étiquettes trouvées par notre modèle, on remarque une surabondance d'émotion 0 avec les autres émotions quasiment non existantes.Cet état n'est probablement pas dû à DBSCAN, mais à notre inhabileté à faire marcher l'algorithme à dataset.
Nous avons choisi DBSCAN après avoir lu cet article (http://fse.studenttheses.ub.rug.nl/18064/1/Report_research_internship.pdf) qui met DBSCAN et Mean Shift dans une lumière favorable pour le clustering de visages similaires. 

# Partie Final: Conclusion

##### À faire (SVM et SVR):

#
<font color=magenta>
1. Décrivez la méthode de normalisation de données utilisée.<br>
</font>
 Nous utilisons MinMaxScaler de la librairie scikit learn pour faire la normalisation de données entre des valeurs de 0 et 1, Cet classe met à l'échelle et traduit chaque caractéristique individuellement de telle sorte qu'elle se trouve dans la plage donnée sur l'ensemble de données choisis.

#
<font color=magenta>
2. Présentez brièvement la méthode que vous avez utilisée afin de trouver le meilleur modèle SVM. Quels ont été vos résultats ? Quels sont les impacts des hyperparamètres et leur utilité respective?<br>
</font>
Nous avons choisi d'y aller avec le GridSearch de la librairie hypopt pour faire des tests avec les différents hyperparamètres dans les modèles SVM, même si cela à pris du temps à faire les calculs, ce sont les paramètres qui permettent d'améliorer la précision des modèles.

#
<font color=magenta>
3. Globalement, prenant en compte les trois TPs, quel type d’approche recommanderiez-vous pour les ensembles de données FER et FG-NET et dans quelles conditions (par exemple, mais non exhaustif, le nombre de données privilégié, les hyperparamètres, le temps de calcul, le matériel nécessaire, les scores de performance)? Discutez des performances que vous avez obtenues entre tous les modèles d’apprentissage. <br>
</font>

À travers ces trois TPs, le modèle qui a le mieux performé est notre CNN à sept couches cachées. En effet, avec le data augmentation, nous avions une précision d'un peu de 70% pour FER et FG-NET. Cela est nettement supérieur à toutes les autres modèles vues dans nos laboratoires.  Pour l'entrainer, il nous fallait peu de prétraitement des données et aucune extraction de primitives. Les seuls facteurs limitants sont le matériel nécessaire pour entrainer ce modèle. En effet, c'est seulement grâce aux GPUs de l'école que nous avons pu aller à sept couches. Or, en tenant compte des résultats obtenus. Il est clair que le CNN est le meilleur choix.

Cela étant dit, nous allons parler des modèles les plus pertinents de chaque laboratoire. Dans le laboratoire 1, avec notre meilleur modèle KNN, nous avons eu une précision de 27% avec qui est très bas relative à notre CNN. Cette précision a été trouvée après avoir effectué un gridSearch pour trouver les meilleurs hyperparametres. Nous avions aussi extrait des primitives avec SIFT et LBP. Ces vecteurs extraits ne fournissaient pas des primitives discriminantes, comme il est possible de voir avec les graphiques tracés avec ces derniers. Cela a contribué à faible performance. 

 Dans le laboratoire 2, nous avons travaillé avec des algorithmes de régression linéaires. La régression logistique du dataset FER a produit la précision la plus élevée avec 36%. Lorsqu'on a utilisé les vecteurs SIFT et LBP, cette précision à baissé de 10%. Cela prouve encore une fois que les vecteurs extraits ne sont pas discriminants. Ensuite nous avons testé un MLP et encore une fois en utilisant le dataset FER au complet, nous avons une meilleure précision qu'en utilisant les vecteurs de primitives; 29% v.s 15 à 17%. Le meilleur résultat avec les vecteurs de primitives extraites viennent de PCA LBP avec 24.49%.



#
<font color=magenta>
4. Formulez quelques pistes d’amélioration des classificateurs.<br>
</font>

Pour améliorer la plupart des modèles, il faut trouver des primitives discriminantes. Comme il est possible de voir avec toutes les autres modèles dans les laboratoires précédentes, la performance des modèles dépendent énormément de l'information transmise, soit l'adage Garbage In, Garbage out. Si nous avions pu trouver dans le laboratoire 1 des primitives discriminantes de qualité, la performance de toutes les modèles auraient pû être améliorée.  
    
##### À faire (Combinaison de modèles):
#
<font color=magenta>
1. Présentez la conception de votre solution reposant sur la théorie des ensembles. Présentez ici le diagramme nécessaire afin de présenter convenablement votre combinaison de modèles. Faites une discussion expliquant vos décisions de conception.<br>
</font>
Utilisation du VotingClassifier de la librairie scikit-learn pour faire la combinaison de modèles selon deux différentes règles soit __le vote par majorité__ et __la votation pondérée sur les classes prédites__, le jeu de primitives PCA_lbp est le vecteur de données pour faire l'évaluation de ce classificateur par votation avec quatre différents modèles de classification, soit le DecisionTreeClassifier, le randomForestClassifier, le gaussian naives bayes et KNeighborsClassifier.

|JEU DE PRIMITIVES| CLASSIFICATEUR |COMBINAISON |RÉSULTATS|
|-|-|-|-|                   
||                   ---->DT------>
||                   ---->RT------>
|PCA_lbp -------->|================|--->VotingClassifier--->|y
||                   ---->NB------>
||                   ---->nKK----->

|JEU DE PRIMITIVES| CLASSIFICATEUR |COMBINAISON |RÉSULTATS|
|-|-|-|-|                   
||                   ---->DT------>(1.8)
||                   ---->RT------>(0.8)
|PCA_lbp -------->|================|--->VotingClassifier--->|y
||                   ---->NB------>(1.2)
||                   ---->nKK----->(0.8)


#
<font color=magenta>
2. Est-ce que la combinaison de modèles a apporté des améliorations dans les performances? Comparer la combinaison de modèles avec les modèles individuels.<br>
</font>
La performance par les six classificateurs individuels de la partie 1 sont moins élevé avec des précisions obtenues entre 16% et 18% pour l'ensemble validation et l'ensemble test, tandis que les ensembles à l'aide de VotingClassifier obtiennent au moins une précision de 22% et 24% pour les ensembles de validation et de test, ce qui représente une différence d'au moins 4% entre les classificateurs simples et des ensembles avec VotingClassifier. 

    

##### À faire (Regroupement): 
<font color=magenta>    
1. Présentez vos conclusions sur les algorithmes de regroupement. Sont-ils vraiment utiles? Ont-ils produit des résultats cohérents? Est-ce que c’est facile à analyser les sorties (groupes) et d'en tirer des conclusions sur quelles étiquettes assigner à ces groupes?<br>
</font>

    
    

Théoriquement, les émotions et leur caractéristiques sont distinctes. Dans cette mesure, il doit être possible de créer des clusters capable de diviser l'informations en clusters. Cependant, avec notre dataset FER, il n'est possible de dire que les algorithmes de regroupement sont utiles.  Les données ne semblent pas être capables d'être regroupées en clusters alors les deux algorithmes utilisés ne fonctionnent pas. Les clusters créés ne représentent pas la réalité des données fournies alors les conclusions tirées seront forcément erronées. 